In [17]:
from web3 import Web3, EthereumTesterProvider
from eth_tester import PyEVMBackend
from solcx import install_solc, compile_source
import time
import sys

class OutputTee:
    def __init__(self, filename):
        self.file = open(filename, 'w', encoding='utf-8')
        self.original_stdout = sys.stdout  # 备份原输出流

    def write(self, text):
        # 同时写入文件和控制台
        self.file.write(text)
        self.original_stdout.write(text)

    def flush(self):
        # 确保缓冲区内容写入文件
        self.file.flush()
        self.original_stdout.flush()

sys.stdout = OutputTee('opcode_debug.log')

# 安装指定版本的 Solidity 编译器
print("安装 Solidity 编译器 (solc 0.5.0)...")
install_solc(version='0.5.0')
print("Solidity 编译器安装完成\n")

# 使用 EthereumTesterProvider 创建一个本地的以太坊测试环境
print("创建本地以太坊测试环境...")
w3 = Web3(EthereumTesterProvider(ethereum_tester=PyEVMBackend()))
print("测试环境创建成功\n")

# Solidity 源代码
solidity_source = '''
pragma solidity ^0.5.0;

contract TestAddMultiCost {

    uint256 a;
    uint256 b;
    uint256 c;

    // 构造函数：初始化 a 和 b 的值
    constructor(uint256 x, uint256 y) public {
        a = x;
        b = y;
    }

    // AddMulti 函数：通过循环执行加法和乘法操作以消耗 gas
    function AddMulti() public {
        int j = 10000;
        while(j > 0){
            c = a + b;
            c = a * b;
            j--;
        }
    }

}
'''

print("开始编译 Solidity 合约...")
compiled_sol = compile_source(
    solidity_source,
    output_values=['abi', 'bin'],
    solc_version="0.5.0"
)
print("合约编译成功\n")

# 提取合约接口及字节码
contract_id, contract_interface = compiled_sol.popitem()
bytecode = contract_interface['bin']
abi = contract_interface['abi']

print("合约字节码:")
print(bytecode, "\n")

print("合约 ABI:")
print(abi, "\n")

# 设置默认账户，用于后续交易
w3.eth.default_account = w3.eth.accounts[0]
print("测试环境中的账户列表:")
print(w3.eth.accounts, "\n")

# 创建合约对象
print("构造合约对象...")
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)
print("合约对象构造完成\n")

# 部署合约到测试网络（通过构造函数传入参数 1234 和 4321）
print("开始部署合约...")
tx_hash = Greeter.constructor(1234, 4321).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("合约部署完成，交易回执:")
print(tx_receipt, "\n")

# 使用部署后的合约地址构造合约实例
print("通过部署的地址获取合约实例...")
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)
print("合约实例获取成功\n")

# 调用合约中的 AddMulti 函数，并记录运行时间
print("调用合约中的 AddMulti 函数，开始计时...")
t1 = time.time()
tx_hash = greeter.functions.AddMulti().transact()
t2 = time.time()
print("函数调用完成")
print("平均每次操作的耗时 (秒):")
print((t2 - t1) / 10000, "\n")

# 等待 AddMulti 函数的交易完成
print("等待 AddMulti 函数的交易回执...")
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("交易回执接收，函数执行结束")

sys.stdout = sys.stdout.original_stdout


安装 Solidity 编译器 (solc 0.5.0)...
Solidity 编译器安装完成

创建本地以太坊测试环境...
测试环境创建成功

开始编译 Solidity 合约...
合约编译成功

合约字节码:
608060405234801561001057600080fd5b506040516040806101278339810180604052604081101561003057600080fd5b8101908080519060200190929190805190602001909291905050508160008190555080600181905550505060be806100696000396000f3fe608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063fc5a284c146044575b600080fd5b348015604f57600080fd5b5060566058565b005b600061271090505b6000811315608f5760015460005401600281905550600154600054026002819055508080600190039150506060565b5056fea165627a7a72305820778bf9e37d5c5deb52de7d2b623a496269e6a0b07df32111c1cb0f8c1313e9260029 

合约 ABI:
[{'constant': False, 'inputs': [], 'name': 'AddMulti', 'outputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'name': 'x', 'type': 'uint256'}, {'name': 'y', 'type': 'uint256'}], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'constr

In [ ]:
from web3 import Web3, EthereumTesterProvider
from eth_tester import PyEVMBackend
from solcx import install_solc, compile_source
import time
import sys

class OutputTee:
    def __init__(self, filename):
        self.file = open(filename, 'w', encoding='utf-8')
        self.original_stdout = sys.stdout  # 备份原输出流

    def write(self, text):
        # 同时写入文件和控制台
        self.file.write(text)
        self.original_stdout.write(text)

    def flush(self):
        # 确保缓冲区内容写入文件
        self.file.flush()
        self.original_stdout.flush()

sys.stdout = OutputTee('output.log')

# 安装指定版本的 Solidity 编译器
print("安装 Solidity 编译器 (solc 0.5.0)...")
install_solc(version='0.5.0')
print("Solidity 编译器安装完成\n")

# 使用 EthereumTesterProvider 创建一个本地的以太坊测试环境
print("创建本地以太坊测试环境...")
w3 = Web3(EthereumTesterProvider(ethereum_tester=PyEVMBackend()))
print("测试环境创建成功\n")

# Solidity 源代码
solidity_source = '''
pragma solidity ^0.5.0;

contract TestAddMultiCost {

    uint256 a;
    uint256 b;
    uint256 c;

    // 构造函数：初始化 a 和 b 的值
    constructor(uint256 x, uint256 y) public {
        a = x;
        b = y;
    }

    // AddMulti 函数：通过循环执行加法和乘法操作以消耗 gas
    function AddMulti() public {
        int j = 10000;
        while(j > 0){
            c = a + b;
            c = a * b;
            j--;
        }
    }

}
'''

print("开始编译 Solidity 合约...")
compiled_sol = compile_source(
    solidity_source,
    output_values=['abi', 'bin'],
    solc_version="0.5.0"
)
print("合约编译成功\n")

# 提取合约接口及字节码
contract_id, contract_interface = compiled_sol.popitem()
bytecode = contract_interface['bin']
abi = contract_interface['abi']

print("合约字节码:")
print(bytecode, "\n")

print("合约 ABI:")
print(abi, "\n")

# 设置默认账户，用于后续交易
w3.eth.default_account = w3.eth.accounts[0]
print("测试环境中的账户列表:")
print(w3.eth.accounts, "\n")

# 创建合约对象
print("构造合约对象...")
Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)
print("合约对象构造完成\n")

# 部署合约到测试网络（通过构造函数传入参数 1234 和 4321）
print("开始部署合约...")
tx_hash = Greeter.constructor(1234, 4321).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("合约部署完成，交易回执:")
print(tx_receipt, "\n")

# 使用部署后的合约地址构造合约实例
print("通过部署的地址获取合约实例...")
greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)
print("合约实例获取成功\n")

# 调用合约中的 AddMulti 函数，并记录运行时间
print("调用合约中的 AddMulti 函数，开始计时...")
t1 = time.time()
tx_hash = greeter.functions.AddMulti().transact()
t2 = time.time()
print("函数调用完成")
print("平均每次操作的耗时 (秒):")
print((t2 - t1) / 10000, "\n")

# 等待 AddMulti 函数的交易完成
print("等待 AddMulti 函数的交易回执...")
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
print("交易回执接收，函数执行结束")

sys.stdout = sys.stdout.original_stdout


安装 Solidity 编译器 (solc 0.5.0)...
Solidity 编译器安装完成

创建本地以太坊测试环境...
测试环境创建成功

开始编译 Solidity 合约...
合约编译成功

合约字节码:
608060405234801561001057600080fd5b506040516040806101278339810180604052604081101561003057600080fd5b8101908080519060200190929190805190602001909291905050508160008190555080600181905550505060be806100696000396000f3fe608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063fc5a284c146044575b600080fd5b348015604f57600080fd5b5060566058565b005b600061271090505b6000811315608f5760015460005401600281905550600154600054026002819055508080600190039150506060565b5056fea165627a7a72305820778bf9e37d5c5deb52de7d2b623a496269e6a0b07df32111c1cb0f8c1313e9260029 

合约 ABI:
[{'constant': False, 'inputs': [], 'name': 'AddMulti', 'outputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'name': 'x', 'type': 'uint256'}, {'name': 'y', 'type': 'uint256'}], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'constr

In [16]:
from web3 import Web3, EthereumTesterProvider
from eth_tester import PyEVMBackend
from solcx import install_solc, compile_source
import time
install_solc(version='0.5.0')


w3 = Web3(EthereumTesterProvider(ethereum_tester=PyEVMBackend()))

compiled_sol = compile_source(
    '''
pragma solidity ^0.5.0;

contract TestAddMultiCost {

    uint256 a;
    uint256 b;
    uint256 c;

    constructor(uint256 x, uint256 y) public {
        a = x;
        b = y;
    }

    function AddMulti() public {
        int j=10000;
        while(j > 0){
            c = a + b;
            c = a * b;
            j--;
        }
    }

}
    ''',
    output_values=['abi', 'bin'],
    solc_version="0.5.0"
)

contract_id, contract_interface = compiled_sol.popitem()
bytecode = contract_interface['bin']
print(bytecode)
abi = contract_interface['abi']
print(abi)
w3.eth.default_account = w3.eth.accounts[0]
print(w3.eth.accounts)

Greeter = w3.eth.contract(abi=abi, bytecode=bytecode)

tx_hash = Greeter.constructor(1234, 4321).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

print(tx_receipt)

greeter = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi
)

t1 = time.time()
tx_hash = greeter.functions.AddMulti().transact()
t2 = time.time()
print((t2-t1)/10000)

tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)

608060405234801561001057600080fd5b506040516040806101278339810180604052604081101561003057600080fd5b8101908080519060200190929190805190602001909291905050508160008190555080600181905550505060be806100696000396000f3fe608060405260043610603f576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff168063fc5a284c146044575b600080fd5b348015604f57600080fd5b5060566058565b005b600061271090505b6000811315608f5760015460005401600281905550600154600054026002819055508080600190039150506060565b5056fea165627a7a723058206d2afdb5e404a7fcd2e6aed2e89d2453e891a1f586f2cf925d5cae7ff8ae3ed50029
[{'constant': False, 'inputs': [], 'name': 'AddMulti', 'outputs': [], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'function'}, {'inputs': [{'name': 'x', 'type': 'uint256'}, {'name': 'y', 'type': 'uint256'}], 'payable': False, 'stateMutability': 'nonpayable', 'type': 'constructor'}]
['0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf', '0x2B5AD5c4795c026514f8317c7a215E218DcCD6cF', '0x6813Eb9362372EE